In [1]:
# !pip install --upgrade pandas
# !pip install --upgrade google-api-python-client
# !pip install --upgrade seaborn

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io import gbq
from pandas_gbq import read_gbq
import json
import lightgbm as lgb
import gc
import time
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
# from pyspark.sql import functions as sfunc
# from pyspark.sql import types as stypes

In [ ]:
!ls

BQ_Connect-FTRL.ipynb
catboost_training.json
Catboost_v1-More feat.ipynb
DL_Support_v1.ipynb
ftrl_submission.csv
Kaggle-TalkingData-f0e6b0d0e656.json
KNN_v1.ipynb
learn
learn_error.tsv
LGBM_99.ipynb
LGBM_v1.ipynb
LGBM_v1-More feat.ipynb
LGBM_v2.ipynb
LightGBM.ipynb
meta.tsv
RF_Medium_5.ipynb
sub_lgbm_r_to_python_nocv_150upsample.csv
sub_lgbm_r_to_python_nocv_170upsample_maxdepth.csv
sub_lgbm_r_to_python_nocv.csv
sub_xgb_nocv_150upsample.csv
test.csv
time_left.tsv
train.csv
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
Untitled.ipynb
Valid-data_generation.ipynb
Wordbatch.ipynb
Xgboost.ipynb
XGB_v1.ipynb


In [ ]:
VALIDATE = False

MAX_ROUNDS = 2000
EARLY_STOP = 50
OPT_ROUNDS = 2000

FULL_OUTFILE = 'morefeat_rf_medium_sub.csv'
#VALID_OUTFILE = 'sub_lgbm_r_to_python_withcv_150upsample.csv'
VALID_OUTFILE = 'morefeat_rf_medium_valid.csv'

import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split 
import lightgbm as lgb

path = ''

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

print('load train...')
train_cols = ['ip','app','device','os', 'channel', 'click_time', 'is_attributed']
train_df = pd.read_csv(path+"train.csv", dtype=dtypes, skiprows=range(1,144903891), nrows=40000000,usecols=train_cols)

#train_df = pd.read_csv(path+"train.csv", skiprows=range(1,84903891), nrows=100000000,dtype=dtypes, usecols=train_cols)

load train...


In [ ]:
path1 = '../valid-train_data/'
valid_df = pd.read_pickle(path1+'validation.pkl.gz')

In [ ]:
import gc

len_train = len(train_df)

gc.collect()

print('data prep...')

most_freq_hours_in_test_data    = [4, 5, 9, 10, 13, 14]
middle1_freq_hours_in_test_data = [16, 17, 22]
least_freq_hours_in_test_data   = [6, 11, 15]


def shift_feat(df):
    df = df.sort_values(['ip', 'click_time'])
    df['click_time'] = pd.to_datetime(df.click_time)
    df['ip_time_diff'] = df['click_time'].diff().dt.seconds
    df.loc[df.ip != df.ip.shift(), 'ip_time_diff'] = -1
    df = df.sort_values(['ip', 'app','device','os','channel', 'click_time'])
    df['all_time_diff'] = df['click_time'].diff().dt.seconds
    df.loc[df.channel != df.channel.shift(), 'all_time_diff'] = -1
    df.sort_index(inplace=True)
    return df

def prep_data( df ):
    
    df['hour'] = pd.to_datetime(df.click_time).dt.hour.astype('uint8')
    df['day'] = pd.to_datetime(df.click_time).dt.day.astype('uint8')
    df.drop(['click_time'], axis=1, inplace=True)
    gc.collect()
    
    df['in_test_hh'] = (   4 
                         - 3*df['hour'].isin(  most_freq_hours_in_test_data ) 
                         - 2*df['hour'].isin(  middle1_freq_hours_in_test_data ) 
                         - 1*df['hour'].isin( least_freq_hours_in_test_data ) ).astype('uint8')
    print( df.info() )
    
 
    print('group by : ip_day_test_hh')
    gp = df[['ip', 'day', 'in_test_hh', 'channel']].groupby(by=['ip', 'day',
             'in_test_hh'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_day_test_hh'})
    df = df.merge(gp, on=['ip','day','in_test_hh'], how='left')
    del gp
    df.drop(['in_test_hh'], axis=1, inplace=True)
    print( "nip_day_test_hh max value = ", df.nip_day_test_hh.max() )
    df['nip_day_test_hh'] = df['nip_day_test_hh'].astype('uint32')
    gc.collect()
    print( df.info() )

    print('group by : ip_day_hh')
    gp = df[['ip', 'day', 'hour', 'channel']].groupby(by=['ip', 'day', 
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_day_hh'})
    df = df.merge(gp, on=['ip','day','hour'], how='left')
    del gp
    print( "nip_day_hh max value = ", df.nip_day_hh.max() )
    df['nip_day_hh'] = df['nip_day_hh'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_os')
    gp = df[['ip', 'day', 'os', 'hour', 'channel']].groupby(by=['ip', 'os', 'day',
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_os'})
    df = df.merge(gp, on=['ip','os','hour','day'], how='left')
    del gp
    print( "nip_hh_os max value = ", df.nip_hh_os.max() )
    df['nip_hh_os'] = df['nip_hh_os'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_app')
    gp = df[['ip', 'app', 'hour', 'day', 'channel']].groupby(by=['ip', 'app', 'day',
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_app'})
    df = df.merge(gp, on=['ip','app','hour','day'], how='left')
    del gp
    print( "nip_hh_app max value = ", df.nip_hh_app.max() )
    df['nip_hh_app'] = df['nip_hh_app'].astype('uint16')
    gc.collect()
    print( df.info() )

    print('group by : ip_hh_dev')
    gp = df[['ip', 'device', 'hour', 'day', 'channel']].groupby(by=['ip', 'device', 'day',
             'hour'])[['channel']].count().reset_index().rename(index=str, 
             columns={'channel': 'nip_hh_dev'})
    df = df.merge(gp, on=['ip','device','day','hour'], how='left')
    del gp
    print( "nip_hh_dev max value = ", df.nip_hh_dev.max() )
    df['nip_hh_dev'] = df['nip_hh_dev'].astype('uint32')
    gc.collect()
    print( df.info() )

    df.drop( ['ip','day'], axis=1, inplace=True )
    gc.collect()
    print( df.info() )
    
    return( df )

#---------------------------------------------------------------------------------

In [ ]:
print( "Train info before: ")
print( train_df.info() )
#train_df = shift_feat(train_df)
train_df = prep_data( train_df )
gc.collect()
print( "Train info after: ")
print( train_df.info() )

print("vars and data type: ")
train_df.info()

In [ ]:
metrics = 'auc'
#clf = RandomForestClassifier(n_estimators=25)

rf_params = {
        'n_estimators': '1000',
        'criterion': 'gini',
        #'metric':metrics,
        #'learning_rate': 0.1,
        'max_depth': 4,  # -1 means no limit
        #'subsample': 0.7,  # Subsample ratio of the training instance.
        #'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
        'min_samples_split':100,
        'nthread': -1,
        'verbose': 0,
        'n_jobs':-1,
        'random_state':1234,
        'verbose':1,
        'class_weight':{0:1,1:150}
        #'scale_pos_weight':150 # because training data is extremely unbalanced 
}

target = 'is_attributed'
predictors = ['app','device','os', 'channel', 'hour', 'nip_day_test_hh', 'nip_day_hh',
              'nip_hh_os', 'nip_hh_app', 'nip_hh_dev']
# predictors = ['app','device','os', 'channel', 'hour', 'nip_day_test_hh', 'nip_day_hh',
#               'nip_hh_os', 'nip_hh_app', 'nip_hh_dev','ip_time_diff','all_time_diff']

categorical = ['app', 'device', 'os', 'channel', 'hour']

print(train_df.head(5))

In [ ]:
if VALIDATE:
    
    train_df, val_df = train_test_split( train_df, train_size=.95, shuffle=False)

    print(train_df.info())
    print(val_df.info())

    print("train size: ", len(train_df))
    print("valid size: ", len(val_df))

    gc.collect()

    print("Training...")

    num_boost_round=MAX_ROUNDS
    early_stopping_rounds=EARLY_STOP
    
    xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical
                          )
    del train_df
    gc.collect()

    xgvalid = lgb.Dataset(val_df[predictors].values, label=val_df[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical
                          )
    del val_df
    gc.collect()

    evals_results = {}

    bst = lgb.train(lgb_params, 
                     xgtrain, 
                     valid_sets= [xgvalid], 
                     valid_names=['valid'], 
                     evals_result=evals_results, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10, 
                     feval=None)

    n_estimators = bst.best_iteration

    print("\nModel Report")
    print("n_estimators : ", n_estimators)
    print(metrics+":", evals_results['valid'][metrics][n_estimators-1])
    
    outfile = VALID_OUTFILE
    
    del xgvalid

else:

    print(train_df.info())

    print("train size: ", len(train_df))

    gc.collect()

    print("Training...")

    num_boost_round=OPT_ROUNDS
    
#     xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
#                           feature_name=predictors,
#                           categorical_feature=categorical
#                           )
    
    clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=5, n_jobs=-1, 
                           random_state=1234, verbose=1, warm_start=False, class_weight={0:1,1:150})
    bst = clf.fit(train_df[predictors].values,train_df[target].values)
    del train_df
    gc.collect()

#     bst = lgb.train(lgb_params, 
#                      xgtrain, 
#                      num_boost_round=num_boost_round,
#                      verbose_eval=10, 
#                      feval=None)

    outfile = FULL_OUTFILE
    valid_file = VALID_OUTFILE
#del xgtrain
gc.collect()

In [ ]:
# print('load test...')
test_cols = ['ip','app','device','os', 'channel', 'click_time', 'click_id']
test_df = pd.read_csv(path+"test.csv", dtype=dtypes, usecols=test_cols)

test_df = prep_data( test_df )
valid_df = prep_data( valid_df )

gc.collect()

sub = pd.DataFrame()
sub['click_id'] = test_df['click_id']

print("Predicting...")
a = bst.predict_proba(test_df[predictors].values)
a = pd.DataFrame(a)
a.columns = ['no','yes']
sub['is_attributed'] = a['yes'].values
print("writing...")
sub.to_csv(path1+outfile, index=False, float_format='%.9f')

print("done...")
print(sub.info())

In [ ]:
a = bst.predict_proba(test_df[predictors].values)
a = pd.DataFrame(a)
a.head()

In [ ]:
val = pd.DataFrame()
b = bst.predict_proba(valid_df[predictors].values)
b = pd.DataFrame(b)
b.columns = ['no','yes']
val['is_attributed'] = b['yes'].values    
val.to_csv(path1+valid_file, index=False, float_format='%.9f')

In [ ]:
val.head()

In [ ]:
!ls